# Support Vector Classifier using La Classy
WIP. Requires bug fixes from `@lala/appraisal`.

In [24]:
import { parse } from "jsr:@std/csv@0.218";
import {
  ClassificationReport,
  Matrix,
  useSplit,
} from "jsr:@lala/appraisal@0.7.5";
import {
  GradientDescentSolver,
  hinge,
} from "jsr:@lala/classy@1.2.2";

We first load our dataset `spam.csv`.

In [25]:
const data = parse(Deno.readTextFileSync("../datasets/imdb.csv"));

Skip the first row (header).

In [26]:
data.shift()

[ "content", "label" ]

We can now get the predictor and target variables from the dataset.

In [27]:
const x = data.map((fl, i) => fl[0]);
x.slice(0, 10)

[
  "A very, very, very slow-moving, aimless movie about a distressed, drifting young man.",
  "Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.",
  "Attempting artiness with black & white and clever camera angles, the movie disappointed - became eve"... 86 more characters,
  "Very little music or anything to speak of.",
  "The best scene in the movie was when Gerardo is trying to find a song that keeps running through his"... 6 more characters,
  "The rest of the movie lacks art, charm, meaning... If it's about emptiness, it works I guess because"... 12 more characters,
  "Wasted two hours.",
  "Saw the movie today and thought it was a good effort, good messages for kids.",
  "A bit predictable.",
  "Loved the casting of Jimmy Buffet as the science teacher."
]

In [28]:
const y = new Matrix(data.map((fl) => fl[1] === "positive" ? [1] : [-1]), "f64");
y.slice(0, 10)

0,-1
1,-1
2,-1
3,-1
4,1
5,-1
6,-1
7,1
8,-1
9,1


In [29]:
y.shape

[ 1000, 1 ]

We now split our dataset for training and testing purposes. 

In [30]:
const [[x_train, y_train], [x_test, y_test]] = useSplit(
  { ratio: [7, 3], shuffle: true },
  x,
  y
);
x_train.slice(0, 10)

[
  "A very, very, very slow-moving, aimless movie about a distressed, drifting young man.",
  "Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.",
  "Very little music or anything to speak of.",
  "The best scene in the movie was when Gerardo is trying to find a song that keeps running through his"... 6 more characters,
  "The rest of the movie lacks art, charm, meaning... If it's about emptiness, it works I guess because"... 12 more characters,
  "Wasted two hours.",
  "Saw the movie today and thought it was a good effort, good messages for kids.",
  "A bit predictable.",
  "And those baby owls were adorable.",
  "The Songs Were The Best And The Muppets Were So Hilarious."
]

Our input variables are fully text. In order to convert them to vectors, we use a `TextVectorizer`.

In [31]:
import { SplitTokenizer, CountVectorizer } from "jsr:@lala/appraisal@0.7.5";

const tokenizer = new SplitTokenizer({
  skipWords: "english",
  standardize: { lowercase: true },
});
tokenizer.fit(x_train)
const vec = new CountVectorizer(tokenizer.vocabulary.size)

const X_train = vec.transform(tokenizer.transform(x_train, "f64"));
const X_test = vec.transform(tokenizer.transform(x_test, "f64"));
X_train.slice(0, 10);


TypeError: Cannot read properties of undefined (reading 'shape')

Now that we have prepared our inputs, we can initialize our solver.

We use the `hinge` loss function which is used for binary classification with SVM.

In [ ]:
const solver = new GradientDescentSolver({
  loss: hinge(),
});


We can then train our model using the data we acquired.

Setting the learning rate to a small value is desirable. We are training our model for 100 epochs with 20 minibatches.

In [ ]:
solver.train(X_train, y_train, {
  learning_rate: 0.01,
  epochs: 100,
  n_batches: 20,
  patience: 10
});

The model is trained, now it is time to evaluate its performance on our testing dataset

In [ ]:
const res = solver.predict(X_test)
res.shape

[ 300, 1 ]

In [ ]:
res.row(0)

Float64Array(1) [ 6.378700324070068 ]

Minimizing the `hinge` loss increases the margin between positive and negative samples. We take any positive output as a positive example and negative output as a negative one.

In [ ]:
for (let i = 0; i < res.data.length; i += 1) {
  res.data[i] = res.data[i] > 0 ? 1 : -1
}
res.slice(0, 10)

0,1
1,-1
2,-1
3,-1
4,1
5,1
6,-1
7,1
8,-1
9,-1


Finally, we can generate a classification report based on our results.

In [ ]:
new ClassificationReport(y.data, res.data)

Class,Precision,F1Score,Recall,Support
-1,0.620253164556962,0.2978723404255319,0.196,500
1,0.8695652173913043,0.9302325581395349,1,500
Accuracy,,,0.7315,1000
